In [1]:
from hig_data.coco import COCOStuffGraphPrecomputedDataset
import torch
path = '/Users/rupertmenneer/Downloads/coco_graph_train'
coco_graph = COCOStuffGraphPrecomputedDataset(path)
data = coco_graph[10]
data

Found 118287 complete datapoint in /Users/rupertmenneer/Downloads/coco_graph_train


RelaxedHeteroData(
  image=[1, 8, 32, 32],
  mask=[1, 1, 256, 256],
  mask_path='/Users/rupertmenneer/Downloads/coco_graph_train/00000010_mask.npy',
  image_node={
    x=[1],
    pos=[1024, 2],
  },
  class_node={
    x=[2, 183],
    pos=[2, 2],
    label=[2],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 2] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 1021] }
)

In [ ]:
from torch_geometric.loader import DataLoader as GeoDataLoader
dls = GeoDataLoader(coco_graph, batch_size=32, shuffle=True)
graph_batch = next(iter(dls))
graph_batch

In [ ]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 4, gnn_metadata = graph_batch.metadata())a
hignn = precond.unet.init_hignn
blank_image = torch.zeros((32, 192, 32, 32))
out, new_graph = hignn(blank_image, graph_batch.clone())
out.shape

In [ ]:
from hig_data.visualisation import plot_array_images
i = 5
batch_list = graph_batch.to_data_list()
out_split = torch.cat(torch.chunk(out[i:i+1], 64, dim=1), dim=0)
plot_array_images(out_split.detach())
plot_array_images(batch_list[i].mask)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extract the 2nd column from edge_index
linear_image_ids = data['class_to_image'].edge_index[1]
class_image_ids = data['class_to_image'].edge_index[0]

# Initialize a 32x32 image with zeros
image = np.zeros((32, 32))

# Convert linear indices to 2D indices and set the corresponding pixels to 1
for idx in range(len(linear_image_ids)):
    img_idx = linear_image_ids[idx]
    class_idx = class_image_ids[idx]
    row = img_idx // 32
    col = img_idx % 32
    image[row, col] = class_idx + 1

# Display the image
plt.imshow(image,)

plt.show()

In [ ]:
import PIL
image_data = np.load(data.mask_path)
print(image_data.shape)
plt.imshow(image_data.transpose(1, 2, 0))

In [ ]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape